In [1]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, gaussian_kde
from sklearn.cluster import DBSCAN, MeanShift
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
from scipy.spatial import ConvexHull, cKDTree
import copy
import pandas as pd
import numpy.linalg as LA

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Functions

In [6]:
def create_voxel_grid(point_cloud, voxel_size):
    points_np = np.asarray(point_cloud.points)  # Convert points to a NumPy array
    min_bound = np.min(points_np, axis=0)
    
    # Compute voxel indices for all points at once using vectorized operations
    voxel_indices = np.floor((points_np - min_bound) / voxel_size).astype(int)

    # Create a dictionary mapping each voxel to its list of point indices
    voxel_grid = {}
    for i, voxel_index in enumerate(map(tuple, voxel_indices)):  # tuple to use as dict key
        voxel_grid.setdefault(voxel_index, []).append(i)

    return voxel_grid

def calculate_voxel_centers(point_cloud, voxel_grid):
    points_np = np.asarray(point_cloud.points)  # Convert once outside the loop
    voxel_centers = {}

    for voxel_index, point_indices in voxel_grid.items():
        points = points_np[point_indices]  # Indexing directly from the pre-converted NumPy array
        gravity_center = np.mean(points, axis=0)
        voxel_centers[voxel_index] = gravity_center

    return voxel_centers




In [23]:
def find_correspondences(tls_points, dls_points):
    # Find the nearest neighbors between TLS and DLS points
    nbrs = NearestNeighbors(n_neighbors=1).fit(dls_points)
    distances, indices = nbrs.kneighbors(tls_points)

    # Return the correspondences as a dictionary
    correspondences = {'tls_indices': np.arange(len(tls_points)), 'dls_indices': indices.flatten()}

    return correspondences


def estimate_transformation(correspondences, tls_valid_p, dls_valid_p):
    tls_indices = correspondences['tls_indices']
    dls_indices = correspondences['dls_indices']

    # Use NumPy array indexing instead of list comprehension for better performance
    dls_points_matched = np.array(dls_valid_p)[dls_indices]
    tls_points_matched = np.array(tls_valid_p)[tls_indices]

    # Calculate centroids of the matched point pairs
    centroid_dls = np.mean(dls_points_matched, axis=0)
    centroid_tls = np.mean(tls_points_matched, axis=0)

    # Construct matrix H
    H = np.dot((dls_points_matched - centroid_dls).T, (tls_points_matched - centroid_tls))
    
    # Compute SVD to estimate rotation matrix R
    U, _, Vt = np.linalg.svd(H)
    R = np.dot(Vt.T, U.T)
    
    # Ensure no reflection by checking the determinant
    if np.linalg.det(R) < 0:
        Vt[-1, :] *= -1
        R = np.dot(Vt.T, U.T)
    
    # Calculate translation vector t
    t = centroid_tls - np.dot(R, centroid_dls)

    # Construct transformation matrix
    transformation = np.eye(4)
    transformation[:3, :3] = R
    transformation[:3, 3] = t

    return transformation




def check_convergence(iteration, current_rmse):
    # Check convergence condition based on a threshold or criteria
    # Here's a placeholder example that checks if the registration has converged
    # based on the maximum number of iterations or a threshold difference in RMSE
    max_iterations = 15
    threshold_rmse = 0.01

    # Additional logic to determine convergence based on your criteria
    # You can modify or replace this condition according to your requirements
    if iteration >= max_iterations or current_rmse < threshold_rmse:
        return True
    else:
        return False


def axis_angle_to_matrix(axis, theta):
    #skwed object matrix
    w = np.array([[ 0.0, -axis[2], axis[1]],
                   [axis[2], 0.0, -axis[0]],
                  [-axis[1], axis[0], 0.0]
    ])
    
    rot = np.identity(3) + (np.sin(theta)*w) \
    + ((1-np.cos(theta))*np.dot(w,w))
    
    return rot



def calculate_distances(tls_gravity_points, dls_gravity_points):
    # Calculate distances between TLS and DLS gravity center points in each voxel
    distances = {}
    for voxel_index in tls_gravity_points.keys():
        tls_gravity_center = tls_gravity_points[voxel_index]
        dls_gravity_center = dls_gravity_points.get(voxel_index)

        if dls_gravity_center is not None and dls_gravity_center.size > 0:
            distance = np.linalg.norm(tls_gravity_center - dls_gravity_center)
            distances[voxel_index] = distance
    return distances

In [2]:
def calculate_gravity_centers(voxel_indices, tls_ds_points, dls_ds_points, min_bound):
    tls_voxel_points, dls_voxel_points = {}, {}

    tls_ds_points_np = np.array(tls_ds_points)
    dls_ds_points_np = np.array(dls_ds_points)

    for voxel_index in voxel_indices:
        size = voxel_index[3]
        x_min, x_max = voxel_index[0] * voxel_default_size + min_bound[0], voxel_index[0] * voxel_default_size + min_bound[0] + size
        y_min, y_max = voxel_index[1] * voxel_default_size + min_bound[1], voxel_index[1] * voxel_default_size + min_bound[1] + size
        z_min, z_max = voxel_index[2] * voxel_default_size + min_bound[2], voxel_index[2] * voxel_default_size + min_bound[2] + size
        
        # filter out points
        tls_mask = (tls_ds_points_np[:, 0] >= x_min) & (tls_ds_points_np[:, 0] <= x_max) & \
                   (tls_ds_points_np[:, 1] >= y_min) & (tls_ds_points_np[:, 1] <= y_max) & \
                   (tls_ds_points_np[:, 2] >= z_min) & (tls_ds_points_np[:, 2] <= z_max)
        dls_mask = (dls_ds_points_np[:, 0] >= x_min) & (dls_ds_points_np[:, 0] <= x_max) & \
                   (dls_ds_points_np[:, 1] >= y_min) & (dls_ds_points_np[:, 1] <= y_max) & \
                   (dls_ds_points_np[:, 2] >= z_min) & (dls_ds_points_np[:, 2] <= z_max)

        # calculate gravity points
        if np.any(tls_mask):
            tls_voxel_points[voxel_index] = np.mean(tls_ds_points_np[tls_mask], axis=0)
        if np.any(dls_mask):
            dls_voxel_points[voxel_index] = np.mean(dls_ds_points_np[dls_mask], axis=0)

    return tls_voxel_points, dls_voxel_points


def calculate_gravity_centers_optimized(voxel_indices, tls_points, dls_points, min_bound):

    tls_points = np.array(tls_points) 
    dls_points = np.array(dls_points) 
    sorted_tlsp = tls_points[tls_points[:, 0].argsort()]
    sorted_dlsp = dls_points[dls_points[:, 0].argsort()]

    
    """
    Calculate gravity centers for TLS and DLS points in each voxel and store points in each voxel.

    :param voxel_indices: A set of voxel indices to process, each with xyz coordinates and size.
    :param tls_points: TLS points as a numpy array.
    :param dls_points: DLS points as a numpy array.
    :param min_bound: The minimum bounding coordinates of the point cloud.
    :return: Three objects - two dictionaries containing the gravity centers of TLS and DLS points for each voxel,
             and a dictionary containing points in each voxel.
    """
    tls_voxel_gravity_centers = {}  # Gravity centers for TLS
    dls_voxel_gravity_centers = {}  # Gravity centers for DLS
    vtls_points = {}  # TLS points in each voxel
    vdls_points = {}  # DLS points in each voxel
    
    for voxel_index in voxel_indices:
        size = voxel_index[3]
        x_min, x_max = voxel_index[0] * voxel_default_size + min_bound[0], voxel_index[0] * voxel_default_size + min_bound[0] + size
        y_min, y_max = voxel_index[1] * voxel_default_size + min_bound[1], voxel_index[1] * voxel_default_size + min_bound[1] + size
        z_min, z_max = voxel_index[2] * voxel_default_size + min_bound[2], voxel_index[2] * voxel_default_size + min_bound[2] + size
        
        # find TLS and DLS points in each voxel
        voxel_tls_points = find_points_in_voxel(sorted_tlsp, x_min, x_max, y_min, y_max, z_min, z_max)
        voxel_dls_points = find_points_in_voxel(sorted_dlsp, x_min, x_max, y_min, y_max, z_min, z_max)
        vtls_points[voxel_index] = voxel_tls_points
        vdls_points[voxel_index] = voxel_dls_points


        # Calculate and store gravity centers
        if len(voxel_tls_points) > 0:
            tls_voxel_gravity_centers[voxel_index] = np.mean(voxel_tls_points, axis=0)
        if len(voxel_dls_points) > 0:
            dls_voxel_gravity_centers[voxel_index] = np.mean(voxel_dls_points, axis=0)

    return tls_voxel_gravity_centers, dls_voxel_gravity_centers, vtls_points, vdls_points

def find_points_in_voxel(sorted_points, x_min, x_max, y_min, y_max, z_min, z_max):
    
    sorted_points = sorted_points[sorted_points[:, 0].argsort()]

    # bisection for x coordinate
    x_min_index = np.searchsorted(sorted_points[:, 0], x_min, side='right')
    x_max_index = np.searchsorted(sorted_points[:, 0], x_max, side='left')
    x_range_points = sorted_points[x_min_index:x_max_index]
    x_range_points = x_range_points[x_range_points[:, 1].argsort()]

    # for y
    y_min_index = np.searchsorted(x_range_points[:, 1], y_min, side='right')
    y_max_index = np.searchsorted(x_range_points[:, 1], y_max, side='left')
    y_range_points = x_range_points[y_min_index:y_max_index]
    y_range_points = y_range_points[y_range_points[:, 2].argsort()]

    # for z
    z_min_index = np.searchsorted(y_range_points[:, 2], z_min, side='right')
    z_max_index = np.searchsorted(y_range_points[:, 2], z_max, side='left')
    voxel_points = y_range_points[z_min_index:z_max_index]

    return voxel_points


def determine_confidence(voxel_distances, voxels_with_sizes):
    """
    Determine the confidence of each voxel based on the distance between gravity centers and the voxel size.

    :param voxel_distances: A dictionary with voxel indices as keys and distances as values.
    :param voxels_with_sizes: A set of voxel indices with their sizes, used to calculate the threshold.
    :return: Two sets containing confident and unconfident voxel indices.
    """
    confident_voxels = set()
    unconfident_voxels = set()

    for voxel in voxels_with_sizes:
        distance = voxel_distances.get(voxel, float('inf'))
        threshold = (np.sqrt(3) / 4) * voxel[3]  # threshold can be reset
        #threshold = .5

        if distance <= threshold:
            confident_voxels.add(voxel)
        else:
            unconfident_voxels.add(voxel)

    return confident_voxels, unconfident_voxels



def split_voxels(unconfident_voxels, updated_voxels, voxel_default_size):
    """
    Split the unconfident voxels into smaller voxels and update the voxel set.

    :param unconfident_voxels: A set of unconfident voxel indices.
    :param updated_voxels: The current set of all voxel indices.
    :param voxel_default_size: The current size of each voxel.
    :return: The updated set of all voxel indices and the new set of unconfident voxels.
    """
    unconfident_voxels_copy = set(unconfident_voxels)  # duplicate unconf voxels
    for voxel in unconfident_voxels:
        i, j, k, voxel_size = voxel

        # remove unconf
        updated_voxels.remove(voxel)
        unconfident_voxels_copy.remove(voxel)

        # split unconf
        new_size = voxel_size / 2.0
        size_ratio = new_size / voxel_default_size

        for di in range(2):
            for dj in range(2):
                for dk in range(2):
                    new_voxel_index = (i + di * size_ratio, j + dj * size_ratio, k + dk * size_ratio, new_size)
                    updated_voxels.add(new_voxel_index)

    return updated_voxels, unconfident_voxels_copy

def filter_splitable_voxels(unconfident_voxels, min_voxel_size):
    """
    Filter out unconfident voxels that are small enough and should not be split further.

    :param unconfident_voxels: A set of unconfident voxel indices.
    :param min_voxel_size: The minimum voxel size threshold.
    :return: Two sets, one containing voxels to be split and another containing voxels not to be split.
    """
    splitable_voxels = set()
    non_splitable_voxels = set()

    for voxel in unconfident_voxels:
        if voxel[3] > min_voxel_size:
            splitable_voxels.add(voxel)
        else:
            non_splitable_voxels.add(voxel)

    return splitable_voxels, non_splitable_voxels


def filter_points_by_voxels(tls_points, dls_points, confident_voxels, voxel_default_size, min_bound):
    """
    Filter points from TLS and DLS point clouds that are within the confident voxels.

    :param tls_points: TLS points as a numpy array.
    :param dls_points: DLS points as a numpy array.
    :param confident_voxels: A set of voxel indices that are considered confident.
    :param voxel_default_size: The size of each voxel.
    :param min_bound: The minimum bounding coordinates of the point cloud.
    :return: Two lists containing the filtered points from TLS and DLS point clouds.
    """
    conf_tls_points = []
    conf_dls_points = []

    tls_points_np = np.array(tls_points)
    dls_points_np = np.array(dls_points)

    for voxel_index in confident_voxels:
        size = voxel_index[-1]
        x_min, x_max = voxel_index[0] * voxel_default_size + min_bound[0], voxel_index[0] * voxel_default_size + size + min_bound[0]
        y_min, y_max = voxel_index[1] * voxel_default_size + min_bound[1], voxel_index[1] * voxel_default_size + size + min_bound[1]
        z_min, z_max = voxel_index[2] * voxel_default_size + min_bound[2], voxel_index[2] * voxel_default_size + size + min_bound[2]

        # TLS
        tls_mask = (tls_points_np[:, 0] >= x_min) & (tls_points_np[:, 0] <= x_max) & \
                   (tls_points_np[:, 1] >= y_min) & (tls_points_np[:, 1] <= y_max) & \
                   (tls_points_np[:, 2] >= z_min) & (tls_points_np[:, 2] <= z_max)
        conf_tls_points.extend(tls_points_np[tls_mask].tolist())

        # DLS
        dls_mask = (dls_points_np[:, 0] >= x_min) & (dls_points_np[:, 0] <= x_max) & \
                   (dls_points_np[:, 1] >= y_min) & (dls_points_np[:, 1] <= y_max) & \
                   (dls_points_np[:, 2] >= z_min) & (dls_points_np[:, 2] <= z_max)
        conf_dls_points.extend(dls_points_np[dls_mask].tolist())

    return conf_tls_points, conf_dls_points

def extract_confident_points(vtls_points, vdls_points, confident_voxels):
    """
    Extract confident points from voxel points dictionaries based on confident voxels.

    :param vtls_points: A dictionary containing TLS points in each voxel.
    :param vdls_points: A dictionary containing DLS points in each voxel.
    :param confident_voxels: A set of voxel indices that are considered confident.
    :return: Two numpy arrays containing the filtered points from TLS and DLS point clouds.
    """
    conf_tls_points = []
    conf_dls_points = []

    for voxel_index in confident_voxels:
        tls_points_in_voxel = vtls_points.get(voxel_index, [])
        dls_points_in_voxel = vdls_points.get(voxel_index, [])

        conf_tls_points.extend(tls_points_in_voxel)
        conf_dls_points.extend(dls_points_in_voxel)

    return np.array(conf_tls_points), np.array(conf_dls_points)


def calculate_matched_rmse(source_points, target_points):
    # find the nearest point in the target points of each point in source_points
    nbrs = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(target_points)
    distances, indices = nbrs.kneighbors(source_points)

    # matching
    matched_target_points = target_points[indices.flatten()]

    # RMSE
    return np.sqrt(mean_squared_error(source_points, matched_target_points))


def find_correspondences(tls_points, dls_points):
    # Find the nearest neighbors between TLS and DLS points
    nbrs = NearestNeighbors(n_neighbors=1).fit(dls_points)
    distances, indices = nbrs.kneighbors(tls_points)

    # Return the correspondences as a dictionary
    correspondences = {'tls_indices': np.arange(len(tls_points)), 'dls_indices': indices.flatten()}

    return correspondences



def apply_transformation(points, transformation):
    # Apply transformation to points directly without using .T
    transformed_points = points @ transformation[:3, :3].T + transformation[:3, 3]
    return transformed_points


def visualize_confident_points(tls_points, dls_points):
    """
    Visualize TLS and DLS points that are determined as confident.

    :param tls_points: TLS points.
    :param dls_points: DLS points.
    """
    tls_cloud = o3d.geometry.PointCloud()
    tls_cloud.points = o3d.utility.Vector3dVector(tls_points)
    tls_cloud.paint_uniform_color([1, 0, 0])  # 红色

    dls_cloud = o3d.geometry.PointCloud()
    dls_cloud.points = o3d.utility.Vector3dVector(dls_points)
    dls_cloud.paint_uniform_color([0, 1, 0])  # 绿色

    o3d.visualization.draw_geometries([tls_cloud, dls_cloud])


def generate_alpha_shape(raw_points, alpha):
    # Step 1: Create a PointCloud from the raw TLS point data

    pcd.points = o3d.utility.Vector3dVector(raw_points)

    # Step 2: Generate the alpha shape
    alpha_shape = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, alpha)

    # Step 3: Save the alpha shape as a PLY file
    #o3d.io.write_triangle_mesh("alpha_shape.ply", alpha_shape)

    return alpha_shape



In [ ]:
def point_to_plane_icp_knn(source_points):
    closest_points = []
    closest_triangle_normals = []
    
    # Precompute closest points and their normals
    for point in source_points:
        query_point = o3d.core.Tensor([point], dtype=o3d.core.Dtype.Float32)
        result = scene.compute_closest_points(query_point)
        closest_points.append(result['points'].numpy()[0])
        triangle_idx = result['primitive_ids'].numpy()[0]
        closest_triangle_normals.append(np.asarray(triangles_normals)[triangle_idx])
    
    # Convert to NumPy arrays
    closest_points = np.array(closest_points)
    closest_triangle_normals = np.array(closest_triangle_normals)
    
    # Compute point-to-plane distances
    point2mesh_dist = np.linalg.norm(source_points - closest_points, axis=1)

    # Build the A and b matrices for point-to-plane minimization
    np_pcd_s = np.array(source_points)
    A = np.zeros((6, 6))
    b = np.zeros((6, 1))
    
    for i in range(len(np_pcd_s)):
        xn = np.cross(np_pcd_s[i], closest_triangle_normals[i])
        xn_n = np.hstack((xn, closest_triangle_normals[i])).reshape(-1, 1)
        A += np.dot(xn_n, xn_n.T)
        
        nT = closest_triangle_normals[i].reshape(1, -1)
        p_x = (closest_points[i] - np_pcd_s[i]).reshape(-1, 1)
        b += xn_n * np.dot(nT, p_x)

    u_opt = np.linalg.solve(A, b)
    theta = np.linalg.norm(u_opt[:3])
    w = u_opt[:3].flatten() / theta
    rot = axis_angle_to_matrix(w, theta)
    
    # Construct transformation matrix
    transformation = np.eye(4)
    transformation[:3, :3] = rot
    transformation[:3, 3] = u_opt[3:6].flatten()

    # Calculate mean distance
    distances = np.abs(np.sum((np_pcd_s - closest_points) * closest_triangle_normals, axis=1))
    mean_distance = np.mean(distances)

    return transformation, mean_distance



def iterative_icp_process_pt2pl(tls_p, dls_p, initial_v, tls_ds_points, dls_ds_points, min_bound, voxel_default_size, max_iterations=15):
    # Ensure the point clouds are NumPy arrays
    tls_ds_points = np.array(tls_ds_points)
    dls_ds_points = np.array(dls_ds_points)
    
    # Initialize variables
    converged = False
    transformation = np.eye(4)
    iteration = 0
    updated_voxels = initial_v
    final_confident_tls_points, final_confident_dls_points = [], []
    
    # Initial gravity centers calculation
    tls_gp, dls_gp, vtls_points, vdls_points = calculate_gravity_centers_optimized(updated_voxels, tls_ds_points, dls_ds_points, min_bound)
    print('Initial voxel count TLS:', len(vtls_points.keys()), 'DLS:', len(vdls_points.keys()))
    
    
    while not converged:
        # Calculate distances between gravity centers
        vdist_icp = calculate_distances(tls_gp, dls_gp)
        conf_v_icp, unconf_v_icp = determine_confidence(vdist_icp, updated_voxels)
        print('Confident and unconfident before splitting:', len(conf_v_icp), len(unconf_v_icp))
        
        # Split unconfident voxels if needed
        splitable_voxels, _ = filter_splitable_voxels(unconf_v_icp, MIN_VOXEL_SIZE)
        updated_voxels, _ = split_voxels(splitable_voxels, updated_voxels, voxel_default_size)
        
        # If no confident voxels, continue to next iteration
        if len(conf_v_icp) == 0:
            continue
        
        # Extract confident points
        conf_tlsp_icp, conf_dlsp_icp = extract_confident_points(vtls_points, vdls_points, conf_v_icp)
        conf_tlsp_icp = np.array(conf_tlsp_icp)
        conf_dlsp_icp = np.array(conf_dlsp_icp)
        print('Confident TLS:', len(conf_tlsp_icp), 'Confident DLS:', len(conf_dlsp_icp))
        
        # Perform point-to-plane ICP
        transformation, pt2pl_rmse = point_to_plane_icp_knn(conf_dlsp_icp)
        conf_dlsp_icp = apply_transformation(conf_dlsp_icp, transformation)
        
        # Apply the transformation to DLS point cloud and downsampled DLS points
        dls_ds_points = apply_transformation(dls_ds_points, transformation)
        
        # Recalculate gravity centers for updated point cloud
        tls_gp, dls_gp, vtls_points, vdls_points = calculate_gravity_centers_optimized(updated_voxels, tls_ds_points, dls_ds_points, min_bound)
        
        # Compute RMSE between matched points
        pt2pt_rmse = calculate_matched_rmse(conf_dlsp_icp, conf_tlsp_icp)
        
        # Check convergence based on RMSE
        converged = check_convergence(iteration, pt2pl_rmse)
        
        if converged:
            print('FINISHED. pt2ptRMSE:', pt2pt_rmse, 'pt2plRMSE:', pt2pl_rmse)
            final_confident_tls_points, final_confident_dls_points = conf_tlsp_icp, conf_dlsp_icp
            break
        
        iteration += 1
        print('Iteration:', iteration, 'pt2ptRMSE:', pt2pt_rmse, 'pt2plRMSE:', pt2pl_rmse)

    return transformation, iteration, final_confident_tls_points, final_confident_dls_points


## Implement

### initial voxelization

In [ ]:
# load pcd
tls_raw = o3d.io.read_point_cloud("your path")
dls_raw = o3d.io.read_point_cloud("your path")

In [ ]:
# create initial voxels
voxel_size = .5
voxel_size_ds = 0.001

tls_voxel_grid = create_voxel_grid(tls_raw, voxel_size)
dls_voxel_grid = create_voxel_grid(dls_raw, voxel_size)

# compute gravity center
tls_voxel_centers = calculate_voxel_centers(tls_raw, tls_voxel_grid)
dls_voxel_centers = calculate_voxel_centers(dls_raw, dls_voxel_grid)


In [ ]:
# merge two pcds
merged_cloud = tls_raw + dls_raw

# boundary
min_bound = np.min(np.asarray(merged_cloud.points), axis=0)


### meshing on TLS pcd

In [ ]:
# Assuming tls_voxel_grid and dls_voxel_grid are already computed
tls_keys = set(tls_voxel_grid.keys())
dls_keys = set(dls_voxel_grid.keys())

intersection_keys = tls_keys.intersection(dls_keys)

# Replace "tls_raw.points" with your actual raw TLS point data
tls_alpha_shape = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(tls_raw, alpha=0.05)

# create TLS alpha shape
tls_alpha_shape = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(tls_raw, alpha=0.05)
tls_alpha_shape.compute_triangle_normals()

# Access the triangles and triangle normals
triangles = tls_alpha_shape.triangles
triangles_normals = tls_alpha_shape.triangle_normals

# legacy mesh to Tensor mesh
tls_alpha_t = o3d.t.geometry.TriangleMesh.from_legacy(tls_alpha_shape)
scene = o3d.t.geometry.RaycastingScene()

mesh_id = scene.add_triangles(tls_alpha_t)
mesh_ids = {mesh_id: 'tls mesh'}


### fine registration

In [ ]:
# Initial voxel size
voxel_default_size = voxel_size

# Initialize point cloud data, directly using NumPy arrays (no need to convert to lists)
tls_points = np.asarray(tls_raw.points)
dls_points = np.asarray(dls_raw.points)

# Find the intersection of TLS and DLS voxel centers
intersection_voxels = set(tls_voxel_centers.keys()) & set(dls_voxel_centers.keys())

# Create a new matrix containing the xyz coordinates of each voxel and the initial voxel size
initial_voxels = {(x, y, z, voxel_default_size) for (x, y, z) in intersection_voxels}

# Update gravity centers using calculate_gravity_centers
tls_gp, dls_gp, vtls_points_gp, vdls_points_gp = calculate_gravity_centers_optimized(initial_voxels, tls_points, dls_points, min_bound)

# Compute distances between the gravity centers
vdist = calculate_distances(tls_gp, dls_gp)

# Use vdist directly, no need to create an additional alias


In [ ]:
# Set minimum voxel size
MIN_VOXEL_SIZE = voxel_default_size / (2 ** 4)

# No need for deepcopy unless modifying internal structure
initial_voxels_in = initial_voxels.copy()

# Convert point clouds to NumPy arrays once and reuse
tls_points_np = np.asarray(tls_raw.points)
dls_points_np = np.asarray(dls_raw.points)

# Execute iterative point-to-plane ICP with alpha = 0.1
transformation, num_iterations, confident_tls_points, confident_dls_points = iterative_icp_process_pt2pl(
    tls_points_np, dls_points_np, initial_voxels_in, tls_points, dls_points, 
    min_bound, voxel_default_size, max_iterations=15
)

# Print the transformation results
print("Transformation Matrix:")
print(transformation)
print("Number of Iterations:", num_iterations)

# Visualize confident points after convergence
visualize_confident_points(confident_tls_points, confident_dls_points)
